## Project Parameters

In [1]:
project_name = "k9mail-5.304"
project_root = "Datasets/k9mail/"
src_path = "k9mail-5.304/k9mail/src/main/java/"
graph_path = "k9mail-5.304.json"

(project_name,project_root,src_path,graph_path)

('k9mail-5.304',
 'Datasets/k9mail/',
 'k9mail-5.304/k9mail/src/main/java/',
 'k9mail-5.304.json')

## Imports

In [2]:
import random
import json

## Relationship Operations

In [3]:
def compose(tuples1, tuples2, retain1=False, retain2=False):
    result = set()

    # Create a dictionary to store intermediate values
    intermediate_dict = {}

    # Populate the intermediate dictionary with values from tuples2
    for source2, target2 in tuples2:
        intermediate_dict.setdefault(source2, []).append(target2)

    # Iterate through tuples1 and find transitive pairs
    for source1, target1 in tuples1:
        if target1 in intermediate_dict:
            for target2 in intermediate_dict[target1]:
                result.add((source1, target2))

    # tuples in tuples1 with which 2nd element is not in tuple2's 1st elements
    if retain1:
        result |= {(s1, t1) for s1, t1 in tuples1 if all(t1 != s2 for s2, _ in tuples2)}

    # tuples in tuples2 with which 1st element is not in tuple1's 2nd elements
    if retain2:
        result |= {(s2, t2) for s2, t2 in tuples2 if all(s2 != t1 for _, t1 in tuples1)}

    return result

# Example usage:
t1 = [(1, 2), (2, 3), (2, 4), (3, 4), (3, 98)]
t2 = [(2, 'a'), (3, 'b'), (4, 'c'), (4, 'd'), (3, 'c'), (99, 'c')]

print(compose(t1, t2))
print(compose(t1, t2, retain1=True))
print(compose(t1, t2, retain2=True))
print(compose(t1, t2, retain1=True, retain2=True))


{(1, 'a'), (2, 'd'), (2, 'c'), (3, 'd'), (3, 'c'), (2, 'b')}
{(3, 98), (1, 'a'), (2, 'd'), (2, 'c'), (3, 'd'), (3, 'c'), (2, 'b')}
{(1, 'a'), (99, 'c'), (2, 'd'), (2, 'c'), (3, 'd'), (3, 'c'), (2, 'b')}
{(3, 98), (1, 'a'), (99, 'c'), (2, 'd'), (2, 'c'), (3, 'd'), (3, 'c'), (2, 'b')}


In [4]:
def flip_tuples(tuples):
    return {(b, a) for a, b in tuples}

# Example usage:
t = [(1, 2), (3, 4), ('a', 'b')]
flipped = flip_tuples(t)

print(flipped)


{('b', 'a'), (2, 1), (4, 3)}


## Load javapers output

This is the "detailed" graph, i.e., it contains packages, classes, methods, variables. This is a labeled property graph, so a node has id, label, and properties, while an edge has source, target, type, and properties. In both, a property is an arbitrary key-value pair.

In [5]:
with open(project_root+graph_path, encoding="utf8") as json_file:
    data = json.load(json_file)

# samples: first node and first edge in the dataset, just for illustration of how they look like
data['elements']['nodes'][0], data['elements']['edges'][0]

({'data': {'id': 'com.fsck.k9.activity.compose.RecipientLoader.PROJECTION_CRYPTO_ADDRESSES',
   'properties': {'sourceText': 'private static final java.lang.String[] PROJECTION_CRYPTO_ADDRESSES = new java.lang.String[]{ "address", "uid_address" };',
    'visibility': 'private',
    'simpleName': 'PROJECTION_CRYPTO_ADDRESSES',
    'kind': 'field',
    'metaSrc': 'source code'},
   'labels': ['Variable']}},
 {'data': {'id': 'ac7cba3e978bef0c69a74a9b1b4a1e08',
   'source': 'com.fsck.k9.notification.NotificationController.getAccountName(com.fsck.k9.Account).account',
   'label': 'type',
   'properties': {'weight': 1, 'metaSrc': 'source code'},
   'target': 'com.fsck.k9.Account'}})

Put the data in dictionaries with simpler structure for the sake of further processing.

The nodes are now represented as a dictionary where the key is a tuple of (node_id, node_label) and the value is the node properties.

Similarly, the edges are now represented as a dictionary where the key is a tuple of (source, target, edge_type) and the value is the edge properties.

In [6]:
def cy_to_rels(cy_data):
	nodes = {(node['data']['id'],node['data']['labels'][0]):node['data']['properties'] for node in cy_data['elements']['nodes']}
	edges = {(edge['data']['source'],edge['data']['target'],edge['data']['label']):edge['data']['properties'] for edge in cy_data['elements']['edges']}
	return nodes,edges

nodes,edges = cy_to_rels(data)
(len(nodes), len(edges)), random.sample(nodes.items(), 5), random.sample(edges.items(), 5)

/tmp/ipykernel_133895/1496984421.py:7: DeprecationWarning: Sampling from a set deprecated
since Python 3.9 and will be removed in a subsequent version.
  (len(nodes), len(edges)), random.sample(nodes.items(), 5), random.sample(edges.items(), 5)


((13432, 28061),
 [(('com.fsck.k9.mailstore.LocalPart.getSize()', 'Operation'),
   {'sourceText': 'long getSize();',
    'docComment': '',
    'visibility': 'public',
    'simpleName': 'getSize()',
    'kind': 'method',
    'metaSrc': 'source code'}),
  (('com.fsck.k9.K9.DATABASE_VERSION_CACHE', 'Variable'),
   {'sourceText': '/**\r\n * Name of the {@link SharedPreferences} file used to store the last known version of the\r\n * accounts\' databases.\r\n *\r\n * <p>\r\n * See {@link UpgradeDatabases} for a detailed explanation of the database upgrade process.\r\n * </p>\r\n */\r\nprivate static final java.lang.String DATABASE_VERSION_CACHE = "database_version_cache";',
    'visibility': 'private',
    'simpleName': 'DATABASE_VERSION_CACHE',
    'kind': 'field',
    'metaSrc': 'source code'}),
  (('com.fsck.k9.activity.setup.AccountSetupBasics.onActivityResult(int,int,android.content.Intent).resultCode',
    'Variable'),
   {'simpleName': 'resultCode',
    'kind': 'parameter',
    'metaS

Now I'm going to convert the data (yet again) into "relationships". A relationship R is a set of tuples (A,B) in which A and B are nodes in the graph, and there is an edge of type R from A to B.

First let's see what relationships are there:

In [7]:
rel_types = {typ for (_, _, typ) in edges}
rel_types

{'contains',
 'hasParameter',
 'hasScript',
 'hasVariable',
 'instantiates',
 'invokes',
 'returnType',
 'specializes',
 'type'}

This is the real conversion:

In [8]:
detailed_rels = {rel_type:{(source,target) for (source, target, typ) in edges if typ == rel_type} for rel_type in rel_types}

[(rel_type, len(detailed_rels[rel_type])) for rel_type in detailed_rels]

[('hasParameter', 5132),
 ('hasScript', 4653),
 ('hasVariable', 2984),
 ('specializes', 159),
 ('invokes', 6266),
 ('contains', 925),
 ('returnType', 1983),
 ('type', 5238),
 ('instantiates', 721)]

Some samples of the relationships we get from the dataset:

In [9]:
{rel_type:random.sample(rel,2) for rel_type, rel in detailed_rels.items()}

/tmp/ipykernel_133895/2526004073.py:1: DeprecationWarning: Sampling from a set deprecated
since Python 3.9 and will be removed in a subsequent version.
  {rel_type:random.sample(rel,2) for rel_type, rel in detailed_rels.items()}


{'hasParameter': [('com.fsck.k9.controller.MessagingController.copyMessagesInThread(com.fsck.k9.Account,java.lang.String,java.util.List,java.lang.String)',
   'com.fsck.k9.controller.MessagingController.copyMessagesInThread(com.fsck.k9.Account,java.lang.String,java.util.List,java.lang.String).destFolder'),
  ('com.fsck.k9.preferences.SettingsExporter.writeSettings(org.xmlpull.v1.XmlSerializer,java.util.Map)',
   'com.fsck.k9.preferences.SettingsExporter.writeSettings(org.xmlpull.v1.XmlSerializer,java.util.Map).prefs')],
 'hasScript': [('com.fsck.k9.notification.NotificationActionCreator',
   'com.fsck.k9.notification.NotificationActionCreator.com.fsck.k9.notification.NotificationActionCreator(android.content.Context)'),
  ('com.fsck.k9.Account', 'com.fsck.k9.Account.getChipColor()')],
 'hasVariable': [('com.fsck.k9.activity.UpgradeDatabases',
   'com.fsck.k9.activity.UpgradeDatabases.mBroadcastReceiver'),
  ('com.fsck.k9.view.MessageCryptoDisplayStatus',
   'com.fsck.k9.view.MessageCry

I'm going to generate the abstract graph from the detailed graph above.

For the nodes, I'm going to only take the packages (Containers) and classes (Structures).

In [10]:
containers = {id for (id,label) in nodes if label == 'Container'}
structures = {id for (id,label) in nodes if label == 'Structure'}
containers, random.sample(structures,5)

/tmp/ipykernel_133895/4061672489.py:3: DeprecationWarning: Sampling from a set deprecated
since Python 3.9 and will be removed in a subsequent version.
  containers, random.sample(structures,5)


({'com',
  'com.fsck',
  'com.fsck.k9',
  'com.fsck.k9.account',
  'com.fsck.k9.activity',
  'com.fsck.k9.activity.compose',
  'com.fsck.k9.activity.loader',
  'com.fsck.k9.activity.misc',
  'com.fsck.k9.activity.setup',
  'com.fsck.k9.autocrypt',
  'com.fsck.k9.cache',
  'com.fsck.k9.controller',
  'com.fsck.k9.crypto',
  'com.fsck.k9.fragment',
  'com.fsck.k9.helper',
  'com.fsck.k9.helper.jsoup',
  'com.fsck.k9.mailstore',
  'com.fsck.k9.mailstore.migrations',
  'com.fsck.k9.mailstore.util',
  'com.fsck.k9.message',
  'com.fsck.k9.message.extractors',
  'com.fsck.k9.message.html',
  'com.fsck.k9.message.quote',
  'com.fsck.k9.message.signature',
  'com.fsck.k9.notification',
  'com.fsck.k9.power',
  'com.fsck.k9.preferences',
  'com.fsck.k9.provider',
  'com.fsck.k9.remotecontrol',
  'com.fsck.k9.search',
  'com.fsck.k9.service',
  'com.fsck.k9.setup',
  'com.fsck.k9.ui',
  'com.fsck.k9.ui.compose',
  'com.fsck.k9.ui.crypto',
  'com.fsck.k9.ui.dialog',
  'com.fsck.k9.ui.message',
  

For the edges, I'm using the transitive properties, e.g., the relationship C1 "calls" C2 means: the class C1 "hasScript" S1, S1 "invokes" the script S2, and the class C2 "hasScript" S2.

In [11]:
abstract_rels = dict()

abstract_rels['contains'] = detailed_rels['contains']
abstract_rels['specializes'] = detailed_rels['specializes']
abstract_rels['holds'] = compose(detailed_rels['hasVariable'], detailed_rels['type'])
abstract_rels['accepts'] = compose(compose(detailed_rels['hasScript'], detailed_rels['hasParameter']), detailed_rels['type'])
abstract_rels['returns'] = compose(detailed_rels['hasScript'], detailed_rels['returnType'])
abstract_rels['constructs'] = compose(detailed_rels['hasScript'], detailed_rels['instantiates'])
abstract_rels['calls'] = compose(compose(detailed_rels['hasScript'], detailed_rels['invokes']), flip_tuples(detailed_rels['hasScript']))

[(rel_type, len(abstract_rels[rel_type])) for rel_type in abstract_rels]

[('contains', 925),
 ('specializes', 159),
 ('holds', 848),
 ('accepts', 1107),
 ('returns', 1129),
 ('constructs', 502),
 ('calls', 1351)]

Some samples of the abstracted relationships:

In [12]:
{rel_type:random.sample(rel,2) for rel_type, rel in abstract_rels.items()}

/tmp/ipykernel_133895/70113002.py:1: DeprecationWarning: Sampling from a set deprecated
since Python 3.9 and will be removed in a subsequent version.
  {rel_type:random.sample(rel,2) for rel_type, rel in abstract_rels.items()}


{'contains': [('com.fsck.k9.provider', 'com.fsck.k9.provider.MessageProvider'),
  ('com.fsck.k9.activity.compose',
   'com.fsck.k9.activity.compose.RecipientPresenter$RecipientsChangedListener')],
 'specializes': [('com.fsck.k9.activity.Accounts$PasswordPromptDialog',
   'com.fsck.k9.activity.misc.NonConfigurationInstance'),
  ('com.fsck.k9.mailstore.LocalMimeMessage',
   'com.fsck.k9.mailstore.LocalPart')],
 'holds': [('com.fsck.k9.view.MessageCryptoDisplayStatus',
   'com.fsck.k9.view.MessageCryptoDisplayStatus'),
  ('com.fsck.k9.autocrypt.AutocryptHeaderParser',
   'com.fsck.k9.autocrypt.AutocryptHeaderParser')],
 'accepts': [('com.fsck.k9.helper.Utility', 'java.lang.String'),
  ('com.fsck.k9.preferences.CheckBoxListPreference', 'boolean')],
 'returns': [('com.fsck.k9.helper.Utility', 'java.lang.String'),
  ('com.fsck.k9.activity.FolderInfoHolder', 'int')],
 'constructs': [('com.fsck.k9.preferences.AccountSettings',
   'com.fsck.k9.Account$Searchable'),
  ('com.fsck.k9.activity.Mess

Now moving on to an example of call lifting; for that we focus on relationships "contains" and "calls".

In [13]:
random.sample(abstract_rels['contains'], 10)

/tmp/ipykernel_133895/1344326522.py:1: DeprecationWarning: Sampling from a set deprecated
since Python 3.9 and will be removed in a subsequent version.
  random.sample(abstract_rels['contains'], 10)


[('com.fsck.k9', 'com.fsck.k9.activity'),
 ('com.fsck.k9.message', 'com.fsck.k9.message.PgpMessageBuilder'),
 ('com.fsck.k9.mailstore', 'com.fsck.k9.mailstore.MessageViewInfo'),
 ('com.fsck.k9.mailstore', 'com.fsck.k9.mailstore.LocalPart'),
 ('com.fsck.k9.provider.EmailProvider',
  'com.fsck.k9.provider.EmailProvider$InternalMessageColumns'),
 ('com.fsck.k9.mailstore', 'com.fsck.k9.mailstore.MessageRemovalListener'),
 ('com.fsck.k9.activity.AccountList',
  'com.fsck.k9.activity.AccountList$AccountsAdapter'),
 ('com.fsck.k9.fragment.ConfirmationDialogFragment',
  'com.fsck.k9.fragment.ConfirmationDialogFragment$ConfirmationDialogFragmentListener'),
 ('com.fsck.k9', 'com.fsck.k9.mailstore'),
 ('com.fsck.k9', 'com.fsck.k9.K9$SplitViewMode')]

In [14]:
random.sample(abstract_rels['calls'], 10)

/tmp/ipykernel_133895/2947183328.py:1: DeprecationWarning: Sampling from a set deprecated
since Python 3.9 and will be removed in a subsequent version.
  random.sample(abstract_rels['calls'], 10)


[('com.fsck.k9.mailstore.LocalStore', 'com.fsck.k9.Account'),
 ('com.fsck.k9.activity.compose.RecipientLoader',
  'com.fsck.k9.view.RecipientSelectView$Recipient'),
 ('com.fsck.k9.service.PollService',
  'com.fsck.k9.controller.MessagingController'),
 ('com.fsck.k9.K9', 'com.fsck.k9.Globals'),
 ('com.fsck.k9.notification.DeviceNotifications', 'com.fsck.k9.K9'),
 ('com.fsck.k9.provider.MessageProvider$IdExtractor',
  'com.fsck.k9.mailstore.LocalMessage'),
 ('com.fsck.k9.ui.messageview.MessageCryptoPresenter',
  'com.fsck.k9.view.MessageHeader'),
 ('com.fsck.k9.preferences.SettingsImporter',
  'com.fsck.k9.preferences.Storage'),
 ('com.fsck.k9.provider.AttachmentProvider',
  'com.fsck.k9.provider.AttachmentProvider'),
 ('com.fsck.k9.activity.setup.AccountSetupCheckSettings$CheckAccountTask',
  'com.fsck.k9.activity.setup.AccountSetupCheckSettings')]

In [15]:
# import networkx as nx
# import matplotlib.pyplot as plt

# g = nx.DiGraph()

# g.add_edges_from(abstract_rels['calls'])

# nx.draw_spring(g, with_labels=True)
# plt.savefig("calls.png")


In [16]:
from ipycytoscape import CytoscapeWidget
import networkx as nx
g = nx.DiGraph()
g.add_edges_from(abstract_rels['calls'])
cyto = CytoscapeWidget()
cyto.graph.add_graph_from_networkx(g, directed=True)
display(cyto)

CytoscapeWidget(cytoscape_layout={'name': 'cola'}, cytoscape_style=[{'selector': 'node', 'css': {'background-c…

Here's the code to lift the calls.

In [17]:
# example: lift calls

lifted_calls = compose(
	compose(
		abstract_rels['contains'], 
		abstract_rels['calls']), 
	flip_tuples(abstract_rels['contains']))
len(lifted_calls), lifted_calls

(379,
 {('com.fsck.k9', 'com.fsck.k9'),
  ('com.fsck.k9', 'com.fsck.k9.Account'),
  ('com.fsck.k9', 'com.fsck.k9.K9'),
  ('com.fsck.k9', 'com.fsck.k9.PRNGFixes'),
  ('com.fsck.k9', 'com.fsck.k9.Throttle'),
  ('com.fsck.k9', 'com.fsck.k9.controller'),
  ('com.fsck.k9', 'com.fsck.k9.helper'),
  ('com.fsck.k9', 'com.fsck.k9.mailstore'),
  ('com.fsck.k9', 'com.fsck.k9.power'),
  ('com.fsck.k9', 'com.fsck.k9.preferences'),
  ('com.fsck.k9', 'com.fsck.k9.preferences.GlobalSettings'),
  ('com.fsck.k9', 'com.fsck.k9.provider'),
  ('com.fsck.k9', 'com.fsck.k9.search'),
  ('com.fsck.k9', 'com.fsck.k9.service'),
  ('com.fsck.k9', 'com.fsck.k9.widget.list'),
  ('com.fsck.k9.PRNGFixes', 'com.fsck.k9'),
  ('com.fsck.k9.PRNGFixes', 'com.fsck.k9.PRNGFixes'),
  ('com.fsck.k9.activity', 'com.fsck.k9'),
  ('com.fsck.k9.activity', 'com.fsck.k9.activity'),
  ('com.fsck.k9.activity', 'com.fsck.k9.activity.Accounts'),
  ('com.fsck.k9.activity', 'com.fsck.k9.activity.AlternateRecipientAdapter'),
  ('com.fsck.

In [18]:
from ipycytoscape import CytoscapeWidget
import networkx as nx
g = nx.DiGraph()
g.add_edges_from(lifted_calls)
cyto = CytoscapeWidget()
cyto.graph.add_graph_from_networkx(g, directed=True)
display(cyto)

CytoscapeWidget(cytoscape_layout={'name': 'cola'}, cytoscape_style=[{'selector': 'node', 'css': {'background-c…

Selective lift---only lift specific containers.

In [19]:
def selective_lift(rels, rel, container_ids):
	selected_containments = {(source,target) for (source,target) in rels['contains'] if source in container_ids}
	inner = compose(
		selected_containments, 
		abstract_rels[rel], retain2=True)
	lifted_calls = compose(
		inner, 
		flip_tuples(selected_containments), retain1=True)
	return lifted_calls

lifted_selective = selective_lift(abstract_rels, 'calls', ['com.fsck.k9.activity.compose',
  'com.fsck.k9.activity.loader',
  'com.fsck.k9.activity.misc',
  'com.fsck.k9.activity.setup',
  'com.fsck.k9.helper.jsoup',
  'com.fsck.k9.mailstore.migrations',
  'com.fsck.k9.mailstore.util',
  'com.fsck.k9.message.extractors',
  'com.fsck.k9.message.html',
  'com.fsck.k9.message.quote',
  'com.fsck.k9.message.signature',
  'com.fsck.k9.ui.compose',
  'com.fsck.k9.ui.crypto',
  'com.fsck.k9.ui.dialog',
  'com.fsck.k9.ui.message',
  'com.fsck.k9.ui.messageview',
  'com.fsck.k9.widget.list'])
len(lifted_selective), lifted_selective

(1067,
 {('com.fsck.k9.mailstore.LocalFolder', 'com.fsck.k9.helper.FileHelper'),
  ('com.fsck.k9.activity.compose', 'com.fsck.k9.helper.ContactPicture'),
  ('com.fsck.k9.activity.UpgradeDatabases',
   'com.fsck.k9.service.DatabaseUpgradeService'),
  ('com.fsck.k9.activity.MessageCompose', 'com.fsck.k9.activity.MessageList'),
  ('com.fsck.k9.activity.ChooseIdentity',
   'com.fsck.k9.activity.K9ListActivity'),
  ('com.fsck.k9.activity.setup', 'com.fsck.k9.EmailAddressValidator'),
  ('com.fsck.k9.cache.TemporaryAttachmentStore',
   'com.fsck.k9.helper.FileHelper'),
  ('com.fsck.k9.view.RecipientSelectView',
   'com.fsck.k9.view.RecipientSelectView$TokenListener'),
  ('com.fsck.k9.message.TextBodyBuilder',
   'com.fsck.k9.message.TextBodyBuilder'),
  ('com.fsck.k9.mailstore.LocalStore', 'com.fsck.k9.mailstore.LocalStore'),
  ('com.fsck.k9.activity.Accounts$AccountsImportedDialog',
   'com.fsck.k9.activity.Accounts'),
  ('com.fsck.k9.controller.MessagingControllerCommands$PendingMarkAllAsRe

In [20]:
from ipycytoscape import CytoscapeWidget
import networkx as nx
g = nx.DiGraph()
g.add_edges_from(lifted_selective)
cyto = CytoscapeWidget()
cyto.graph.add_graph_from_networkx(g, directed=True)
display(cyto)

CytoscapeWidget(cytoscape_layout={'name': 'cola'}, cytoscape_style=[{'selector': 'node', 'css': {'background-c…

In [21]:

lifted_selective = selective_lift(abstract_rels, 'calls', containers)
len(lifted_selective), lifted_selective

(212,
 {('com.fsck.k9', 'com.fsck.k9'),
  ('com.fsck.k9', 'com.fsck.k9.controller'),
  ('com.fsck.k9', 'com.fsck.k9.helper'),
  ('com.fsck.k9', 'com.fsck.k9.mailstore'),
  ('com.fsck.k9', 'com.fsck.k9.power'),
  ('com.fsck.k9', 'com.fsck.k9.preferences'),
  ('com.fsck.k9', 'com.fsck.k9.provider'),
  ('com.fsck.k9', 'com.fsck.k9.search'),
  ('com.fsck.k9', 'com.fsck.k9.service'),
  ('com.fsck.k9', 'com.fsck.k9.widget.list'),
  ('com.fsck.k9.activity', 'com.fsck.k9'),
  ('com.fsck.k9.activity', 'com.fsck.k9.activity'),
  ('com.fsck.k9.activity', 'com.fsck.k9.activity.compose'),
  ('com.fsck.k9.activity', 'com.fsck.k9.activity.misc'),
  ('com.fsck.k9.activity', 'com.fsck.k9.activity.setup'),
  ('com.fsck.k9.activity', 'com.fsck.k9.autocrypt'),
  ('com.fsck.k9.activity', 'com.fsck.k9.controller'),
  ('com.fsck.k9.activity', 'com.fsck.k9.fragment'),
  ('com.fsck.k9.activity', 'com.fsck.k9.helper'),
  ('com.fsck.k9.activity', 'com.fsck.k9.mailstore'),
  ('com.fsck.k9.activity', 'com.fsck.k9.

In [22]:
from ipycytoscape import CytoscapeWidget
import networkx as nx
g = nx.DiGraph()
g.add_edges_from(lifted_selective)
cyto = CytoscapeWidget()
cyto.graph.add_graph_from_networkx(g, directed=True)
display(cyto)

CytoscapeWidget(cytoscape_layout={'name': 'cola'}, cytoscape_style=[{'selector': 'node', 'css': {'background-c…

In [23]:
def find_nodes(nodes, id):
	return [(id_,label) for (id_,label) in nodes if id_ == id]

## networkx Stuff

In [24]:
import networkx as nx

g = nx.MultiDiGraph()

for (id, label), properties in nodes.items():
    g.add_node(id, label=label, **properties)

for (source, target, typ), properties in edges.items():
    g.add_edge(source, target, type=typ, **properties)

g.number_of_nodes(), g.number_of_edges()

(13432, 28061)

In [25]:
for node, nbrsdict in list(g.adj.items())[:10]:
    print((node, nbrsdict))
    

('com.fsck.k9.activity.compose.RecipientLoader.PROJECTION_CRYPTO_ADDRESSES', AdjacencyView({}))
('com.fsck.k9.activity.ActivityListener.pendingCommandsProcessing(com.fsck.k9.Account).account', AdjacencyView({'com.fsck.k9.Account': {0: {'type': 'type', 'weight': 1, 'metaSrc': 'source code'}}}))
('com.fsck.k9.provider.DecryptedFileProvider.getUriForProvidedFile(android.content.Context,java.io.File,java.lang.String,java.lang.String).file', AdjacencyView({}))
('com.fsck.k9.view.ToolableViewAnimator.com.fsck.k9.view.ToolableViewAnimator(android.content.Context,android.util.AttributeSet,int).context', AdjacencyView({}))
('com.fsck.k9.activity.Accounts$ImportSelectionDialog.show(com.fsck.k9.activity.Accounts).activity', AdjacencyView({'com.fsck.k9.activity.Accounts': {0: {'type': 'type', 'weight': 1, 'metaSrc': 'source code'}}}))
('com.fsck.k9.view.CryptoModeSelector$CryptoModeSelectorState.SIGN_ONLY', AdjacencyView({'com.fsck.k9.view.CryptoModeSelector$CryptoModeSelectorState': {0: {'type': 

In [26]:
method_sources = {id:properties['sourceText'] for (id,label),properties in nodes.items() if label in ['Operation', 'Constructor']}
method_sources

{'com.fsck.k9.fragment.MessageListFragmentComparators$DateComparator.com.fsck.k9.fragment.MessageListFragmentComparators$DateComparator()': 'DateComparator() {\r\n}',
 'com.fsck.k9.notification.WearNotifications.buildStackedNotification(com.fsck.k9.Account,com.fsck.k9.notification.NotificationHolder)': 'public android.app.Notification buildStackedNotification(com.fsck.k9.Account account, com.fsck.k9.notification.NotificationHolder holder) {\r\n    int notificationId = holder.notificationId;\r\n    com.fsck.k9.notification.NotificationContent content = holder.content;\r\n    android.support.v4.app.NotificationCompat.Builder builder = createBigTextStyleNotification(account, holder, notificationId);\r\n    android.app.PendingIntent deletePendingIntent = actionCreator.createDismissMessagePendingIntent(context, content.messageReference, holder.notificationId);\r\n    builder.setDeleteIntent(deletePendingIntent);\r\n    addActions(builder, account, holder);\r\n    return builder.build();\r\n

## LLM Stuff

In [27]:
# from txtai.pipeline import LLM

# # llm = LLM("Open-Orca/Mistral-7B-OpenOrca")
# # llm = LLM("microsoft/phi-2", trust_remote_code='True')
# llm = LLM("lmsys/vicuna-7b-v1.5")
# # llm = LLM("TinyLlama/TinyLlama-1.1B-Chat-v0.4")
# for method_name, method_src in random.sample(method_sources.items(), 1):
# 	prompt = f"""
# Answer the following question using the provided context.

# Question:
# What does this Java method do?

# Context:
# {method_src}"""
# 	result = llm(prompt)
# 	print(prompt)
# 	print()
# 	print(result)
# 	print()
# 	print()

In [28]:
# method_samples = random.sample(method_sources.items(), 10)

# with open("method_samples.json", "w", encoding="utf8") as outfile:
#     json.dump(method_samples, outfile)

In [1]:
import json

with open("method_samples.json", encoding="utf8") as json_file:
    method_samples = json.load(json_file)

method_samples

[['com.fsck.k9.fragment.MessageListFragment.onExpunge()',
  'public void onExpunge() {\r\n    if (currentFolder != null) {\r\n        onExpunge(account, currentFolder.name);\r\n    }\r\n}'],
 ['com.fsck.k9.provider.EmailProvider.getThreadedMessages(java.lang.String,java.lang.String[],java.lang.String,java.lang.String[],java.lang.String)',
  'protected android.database.Cursor getThreadedMessages(java.lang.String accountUuid, final java.lang.String[] projection, final java.lang.String selection, final java.lang.String[] selectionArgs, final java.lang.String sortOrder) {\r\n    com.fsck.k9.Account account = getAccount(accountUuid);\r\n    com.fsck.k9.mailstore.LockableDatabase database = getDatabase(account);\r\n    try {\r\n        return database.execute(false, new com.fsck.k9.mailstore.LockableDatabase.DbCallback<android.database.Cursor>() {\r\n            @java.lang.Override\r\n            public android.database.Cursor doDbWork(android.database.sqlite.SQLiteDatabase db) throws com.fs

In [2]:
from llmplus import LlmFactory

# Load the model from Huggingface
model = LlmFactory("Open-Orca/Mistral-7B-OpenOrca")

# Create a llm
llm = model(temperature=0, max_new_tokens=4096, stop=['\n'])

for method_name, method_src in method_samples:
	prompt = f"""Instruction:
Answer the following question using the provided context.

Question:
What does this Java method do?

Context:
{method_src}

Answer:
This method"""
	result = llm(prompt)
	print(prompt)
	print(result)
	print()
	print('==========')

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

/home/srukmono/Code/dev-assist/.venv39/lib/python3.9/site-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The function `__call__` was deprecated in LangChain 0.1.7 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:32000 for open-end generation.
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:32000 for open-end generation.


Instruction:
Answer the following question using the provided context.

Question:
What does this Java method do?

Context:
public void onExpunge() {
    if (currentFolder != null) {
        onExpunge(account, currentFolder.name);
    }
}

Answer:
This method
 is called "onExpunge" and it checks if the current folder is not null. If it's not null, it calls another method named "onExpunge" with two parameters: account and the name of the current folder.



The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:32000 for open-end generation.


...

Instruction:
Answer the following question using the provided context.

Question:
What does this Java method do?

Context:
public synchronized boolean setFolderPushMode(com.fsck.k9.Account.FolderMode pushMode) {
    com.fsck.k9.Account.FolderMode oldPushMode = folderPushMode;
    folderPushMode = pushMode;
    return pushMode != oldPushMode;
}

Answer:
This method
 sets the folder push mode for an account and returns true if the new folder push mode is different from the previous one.



The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:32000 for open-end generation.


Instruction:
Answer the following question using the provided context.

Question:
What does this Java method do?

Context:
SubjectExtractor() {
}

Answer:
This method
 is called SubjectExtractor and it doesn't have any code or functionality implemented within it. It is an empty method that simply declares its existence in the class.



The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:32000 for open-end generation.


Instruction:
Answer the following question using the provided context.

Question:
What does this Java method do?

Context:
public void showCcUncompletedError() {
    ccView.setError(ccView.getContext().getString(R.string.compose_error_incomplete_recipient));
}

Answer:
This method
 displays an error message in the view when the recipient information is incomplete.

Instruction:
Answer the following question using the provided context.

Question:
What does this Java method do?

Context:
@java.lang.Override
public boolean retain() {
    if (mDialog != null) {
        mDialog.dismiss();
        mDialog = null;
        return true;
    }
    return false;
}

Answer:
This method
 overrides the retain() method in the java.lang package and is used to dismiss a dialog box if it exists, setting it to null, and then returning true. If no dialog box exists, it returns false.

